In [31]:
import glob, os, re
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split

In [32]:
txt_path = 'sentences_for_word2vec.txt'
raw_corpus = []

with open(txt_path, 'r', encoding = 'utf-8') as f:
    raw = f.read().splitlines()
    raw_corpus.extend(raw)
print(len(raw_corpus))

125702


In [33]:
raw_corpus[:5]

['무얼 믿은 걸까 부족했던 내게서',
 '나조차 못 믿던 내게 여태 머문 사람',
 '무얼 봤던 걸까 가진 것도 없던 내게',
 '무작정 내 손을_잡아 날 이끈 사람',
 '최고였어']

In [34]:
print(len(max(raw_corpus, key = len)))

423


In [35]:
#raw_corpus = ' '.join(raw_corpus).split()
#raw_corpus = list(filter(None, raw_corpus))
print(len(raw_corpus))
print(raw_corpus[:5])

125702
['무얼 믿은 걸까 부족했던 내게서', '나조차 못 믿던 내게 여태 머문 사람', '무얼 봤던 걸까 가진 것도 없던 내게', '무작정 내 손을_잡아 날 이끈 사람', '최고였어']


In [36]:
print(raw_corpus[-5:])

['찾아_나설까', '사람아 사람아', '내 하나의_사람아', '이_늦은 참회-를', '너는_아는 지']


In [37]:
def preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z가-힣?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = re.sub(r"\(.\)", " ", sentence) # 7
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

In [38]:
print(preprocess_sentence("This @_is ;;;sample        (sentences) sentence . 그리고 "))

<start> This is sample sentences sentence . 그리고 <end>


In [39]:
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    if sentence[-1] == ':':
        continue
    if len(sentence)>150:
        continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
print(len(corpus))
print(corpus[:10])

125685
['<start> 무얼 믿은 걸까 부족했던 내게서 <end>', '<start> 나조차 못 믿던 내게 여태 머문 사람 <end>', '<start> 무얼 봤던 걸까 가진 것도 없던 내게 <end>', '<start> 무작정 내 손을 잡아 날 이끈 사람 <end>', '<start> 최고였어 <end>', '<start> 그대 눈 속에 비친 내 모습 <end>', '<start> 이제는 내게서 그댈 비춰줄게 <end>', '<start> 궂은 비가 오면 <end>', '<start> 세상 가장 큰 그대 우산이 될게 <end>', '<start> 그댄 편히 걸어가요 <end>']


In [40]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 12000, filters = ' ', oov_token = '<unk>')
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = 'post')
    print('토크나이저: ', tokenizer, '\n', tensor)
    
    return tensor, tokenizer

In [41]:
tensor, tokenizer = tokenize(corpus)

토크나이저:  <keras.src.legacy.preprocessing.text.Tokenizer object at 0x0000018D70D82E00> 
 [[    2   736  4362 ...     0     0     0]
 [    2  6889    26 ...     0     0     0]
 [    2   736 10864 ...     0     0     0]
 ...
 [    2     4  1590 ...     0     0     0]
 [    2     7  1548 ...     0     0     0]
 [    2    87   910 ...     0     0     0]]


In [42]:
tensor.shape

(125685, 35)

In [43]:
tensor[0]

array([   2,  736, 4362, 1086, 7539, 3036,    3,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0])

In [46]:
for idx in tokenizer.index_word:
    print(idx, ':', tokenizer.index_word[idx])
    if idx>=1000: break

1 : <unk>
2 : <start>
3 : <end>
4 : 내
5 : 그
6 : 사랑
7 : 이
8 : 나는
9 : 내가
10 : 그대
11 : 날
12 : 나를
13 : 수
14 : 아
15 : 사람
16 : 한
17 : 나
18 : 나의
19 : 난
20 : 다
21 : 우리
22 : 다시
23 : 당신
24 : 없는
25 : 사랑을
26 : 못
27 : 또
28 : 왜
29 : 마음
30 : 저
31 : 너를
32 : 좋아
33 : 가슴에
34 : 사랑이
35 : 거야
36 : 너무
37 : 정말
38 : 더
39 : 사랑은
40 : 사랑의
41 : 당신이
42 : 모두
43 : 가는
44 : 인생
45 : 당신은
46 : 때
47 : 내게
48 : 같은
49 : 이제
50 : 사랑아
51 : 없이
52 : 당신을
53 : 눈물
54 : 가슴
55 : 누가
56 : 보고
57 : 너
58 : 밤
59 : 세월
60 : .
61 : 하나
62 : 말
63 : 이렇게
64 : 눈물이
65 : 함께
66 : 있어요
67 : 세상
68 : 말아요
69 : 너의
70 : 없어
71 : 가사
72 : 영원히
73 : 여자
74 : 바람
75 : 마음을
76 : 두
77 : 가슴이
78 : 오늘도
79 : 이제는
80 : 할
81 : 나도
82 : 안
83 : 게
84 : 한번
85 : 당신의
86 : 잘
87 : 너는
88 : 길
89 : 사람아
90 : 걸
91 : 것
92 : 사랑에
93 : 있는
94 : 두고
95 : 오늘
96 : 줄
97 : 사랑해
98 : 속에
99 : 해도
100 : 좋은
101 : 몰라
102 : 되어
103 : 위해
104 : 그런
105 : 번
106 : 해
107 : 남자
108 : 이젠
109 : 그냥
110 : 그대는
111 : 사랑해요
112 : 모든
113 : 그렇게
114 : 널
115 : 사는
116 : 울고
117 : 언제나
118 : 있어
119 : !
120 : 그대를
121 : 엄마
122 : 때문에
123

In [45]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]
print('텐서 길이: ', tensor.shape)
print('소스문장 길이: ', len(src_input[0]))
print('타겟문장 길이: ', len(tgt_input[0]))

텐서 길이:  (125685, 35)
소스문장 길이:  34
타겟문장 길이:  34
